# A first look at data

date: 12/04/2018

Example code for reading and checking data


In [ ]:
import pandas as pd
import dask.dataframe as dd
import glob as glob
import numpy as np
from scipy.stats import kde

import matplotlib.pyplot as plt
%matplotlib inline 
from matplotlib.colors import LogNorm
plt.rc('font', size=16)
plt.rcParams['figure.figsize'] = (12.0, 10.0)    # resize plots

In [ ]:
filename = '/data/IceCube/2015/output_126836_IC86.2015.hdf'
dataGen = pd.read_hdf(filename, key='Laputop', columns=['x'])
dataGen.drop(dataGen.columns.difference(['x']), 1, inplace=True)
print(dataGen)

In [ ]:
def merge_and_select_data(filenames, output, number_of_files=10000):
    store =pd.HDFStore(output,complib='blosc')
    for i, file in enumerate(filenames):
        print(i, file)
        s = pd.HDFStore(file)
        dataGen = s.select('Laputop')
        dataGen.drop(dataGen.columns.difference(['Run', 'Event','x','y','z','zenith','azimuth','time']), 1, inplace=True)
        dataRec = s.select('LaputopParams')
        dataRec.drop(dataRec.columns.difference(['s125','beta','chi2','ndf']), 1, inplace=True)
        dataCuts = s.select('IT73AnalysisIceTopQualityCuts')
        dataCuts.drop(['Run', 'Event','SubEvent','SubEventStream'], 1, inplace=True)
        dataGen = dataGen.dropna(axis=1, how='all')
        dataRec = dataRec.dropna(axis=1, how='all')
        data = pd.concat([dataGen, dataRec, dataCuts], axis=1, join_axes=[dataGen.index])
        data = data.query('exists!=0 & IceTopMaxSignalInside!=0 & IceTop_reco_succeeded!=0'
                         ' & IceTop_StandardFilter!=0 & IceTopMaxSignalInside !=0 '
                         ' & Laputop_FractionContainment!=0 & BetaCutPassed!=0 &s125>1')
        data.drop(data.columns.difference(['Run','Event','x','y','z','zenith','azimuth',
                                          's125','beta','chi2','ndf']), axis=1,inplace=True)
    
        store.append(key ='data', value=data,  format='t',chunksize=200000)
        s.close()
        if i>number_of_files:
            break

    return store

In [ ]:
filenames = glob.glob('/data/IceCube/2015/*')

number_of_files = 10000
store = merge_and_select_data(filenames, '/data/IceCube/merged2015.h5', number_of_files )        
dataAll = pd.DataFrame() 
dataAll = store.select('data')
print(dataAll.count())
store.close()    




In [ ]:
#print(dataCuts.head())
#print(dataGen.head())
#print(dataRec.head())


In [ ]:
hist = plt.hist(np.log10(dataAll.s125), bins=150)
#plt.hist(np.log10(df.s125), 'ro')